In [ ]:
import streamlit as st
import google.generativeai as genai
from pathlib import Path
from io import BytesIO
from PIL import Image
from api_key import api_key

# Configure Gen AI
genai.configure(api_key=api_key)

# Adjusted generation configuration
generation_config = {
    "temperature": 0.7,  # Lower temperature for more focused output
    "top_p": 0.9,        # Reduced top_p to limit response length
    "top_k": 50,         # Limit the number of choices to consider
    "max_output_tokens": 2048,  # Reduce max tokens to control output length
    "response_mime_type": "text/plain",
}

# Apply safety settings
safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
]

system_prompt = """
You are a highly skilled telecommunication expert specializing in the analysis of output of  "Explainable AI" techniques for improving customer retention purposes, tasked with understanding any if-then rules or feature importance. 
Your primary objective is to carefully review the provided Image  and:
1. if image is SHAP waterfall image Identify any potential impact of features or general interpretation based on the provided data.
s or recommendations for customer retention purposes.
Important notes:
- There are some features "tenure","partner" and "dependents" that are non-actionable, can suggest any sales strategy like cross-selling, up-selling than for customer retention strategy.
- Do not make any assumptions or recommendations that are not supported by the data.
- If no rules are identified, include a statement indicating that no rules were detected.
- Include the following disclaimer in your analysis: "This analysis is intended as a preliminary review. Discuss with your data science team and business operation team before making any strategic or business decisions. Further analysis may be required to confirm these findings."
"""

# Function to load Google Gemini Pro Vision API and get response
model = genai.GenerativeModel(model_name="gemini-1.5-flash",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

# Initialize Streamlit app
st.set_page_config(page_title="Retention Strategy generation", page_icon=":robot")

st.title("Customer Retention Strategy Generation")
st.subheader("An application that helps users to identify visual patterns and recommend retention strategy.")

# File uploader for brain image
uploaded_file = st.file_uploader("Choose your image for analysis...", type=["jpg", "jpeg", "png", "tiff"])

submit = st.button("Generate the Analysis")

# If submit button is clicked and file is uploaded
if uploaded_file:
    # Open the uploaded image
    img = Image.open(uploaded_file)
    
    # Convert the image to RGB mode if it has an alpha channel (RGBA)
    if img.mode == 'RGBA':
        img = img.convert('RGB')
    
    # Resize to 512x512
    img = img.resize((512, 512))
    
    # Save the image into a buffer
    buffer = BytesIO()
    img.save(buffer, format="JPEG")
    image_data = buffer.getvalue()

    # Display the resized image in the app
    st.image(img, caption="Uploaded Image (Resized)", use_column_width=True)

    if submit:
        image_parts = [
            {
                "mime_type": uploaded_file.type,  # Get the mime type of the uploaded file
                "data": image_data
            }
        ]

        prompt_parts = [
            image_parts[0],
            system_prompt,
        ]

        # Generate a response based on the prompt image
        try:
            response = model.generate_content(prompt_parts)
            st.write(response.text)
        except Exception as e:
            st.error(f"An error occurred: {e}")


2025-03-20 21:58:26.606 
  command:

    streamlit run c:\Users\LENOVO\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
